# Burgers' Equation

We will solve a Burgers' equation:

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = v\frac{\partial^2 u}{\partial x^2}, \qquad \text{where} \quad x  \in [-1,1],  \quad t \in [0,1],
$$

with the Dirichlet boundary conditions and the initial condition:

$$
u(-1,t) = u(1,t) = 0, \qquad u(x,0) = -sin(\pi x)
$$

where $v = 0.1/\pi$.

The reference solution is computed by finite differences *here*.

## Implementation and Training

First, we import the libraries:

In [11]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import datetime
import math

We set the value of $v$, the value of $\pi$, define the $sine$ function and the maximum and minimum values of the $x$ domain:

In [12]:
sin = torch.sin
pi = math.pi
v=0.1/pi

x_min = -1.
x_max = 1.

Now, we set the parameters of the neural network: it has a structure with 2 inputs ($x, t$) and 1 output ($u(x,t)$), 20000 maximum training steps, 3 hidden layers with 64 neurons each, 100 samples and a target minimum loss value of $10^{-4}$.

In [13]:
inputs = 2
outputs = 1
hn_1 = 64
hn_2 = 64
hn_3 = 64
steps = 0
max_steps = 20000
loss = 10
min_loss = 1e-4
log_each = 500
samples = 100
loss_values = []

In this part, we define a new class implementing the activation function $sin(x)$, due to the oscillatory nature of the solution.

In [14]:
class Sine(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.sin(x)

The multilayer perceptron (MLP) structure is:

In [15]:
mlp = nn.Sequential(
    nn.Linear(inputs,hn_1),
    Sine(),
    nn.Linear(hn_1, hn_2),
    Sine(),
    nn.Linear(hn_2, hn_3),
    Sine(),
    nn.Linear(hn_3, outputs)
)

optimizer = torch.optim.Adam(mlp.parameters())
criterion = nn.MSELoss()
mlp.train()

Sequential(
  (0): Linear(in_features=2, out_features=64, bias=True)
  (1): Sine()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): Sine()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): Sine()
  (6): Linear(in_features=64, out_features=1, bias=True)
)

In this section, we define a function that calculates the gradients.

In [16]:
def computeGrads(y, x):
    grads, = torch.autograd.grad(y, x, grad_outputs=y.data.new(y.shape).fill_(1), create_graph=True, only_inputs=True)
    return grads

Next, we define the main training loop and the timer:

In [ ]:
starttime_train = datetime.datetime.now()
print('----Training Started----')

while steps < max_steps and loss > min_loss:
    x = (x_max - x_min)*torch.rand(samples) + x_min
    t = torch.rand(samples)
    X = torch.stack([x, t], axis=-1)
    X.requires_grad = True
    Y = mlp(X)
    grads = computeGrads(Y, X)
    dudx = grads[:, :1]
    dudt = grads[:, 1:]
    grads2 = computeGrads(dudx, X)
    d2udx2 = grads2[:, :1]
    pde_loss = criterion(dudt, v * d2udx2 - Y * dudx)

    #initial condition
    t0 = torch.zeros(samples)
    X_initial = torch.stack([x, t0], axis=-1)
    Y_initial = mlp(X_initial)
    u_initial = -sin(pi * x).unsqueeze(1)
    ic_loss = criterion(Y_initial, u_initial)

    #boundary conditions x=-1
    x_1 = -torch.ones(samples)
    X_boundary_1 = torch.stack([x_1, t], axis=-1)
    Y_boundary_1 = mlp(X_boundary_1)
    zero = torch.zeros(samples, 1)
    bc_1_loss = criterion(Y_boundary_1, zero)
    
    #boundary conditions x=1
    x1 = torch.ones(samples)
    X_boundary1 = torch.stack([x1, t], axis=-1)
    Y_boundary1 = mlp(X_boundary1)
    bc1_loss = criterion(Y_boundary1, zero)

    bc_loss = bc_1_loss + bc1_loss

    optimizer.zero_grad()
    loss = pde_loss + ic_loss  + bc_loss
    loss.backward()
    optimizer.step()

    loss_values.append(loss.detach().numpy())

    if steps % log_each == 0:
        print(f'Step:{steps}| pde_loss {pde_loss.item():.5f} ic_loss {ic_loss.item():.5f} bc_loss {bc_loss.item():.5f}')

    steps+=1

endtime_train = datetime.datetime.now()
train_time = endtime_train - starttime_train
train_time_formatted = train_time.seconds + train_time.microseconds / 1e6
print('---Training Finished---')

print(f'Training Duration: {steps} steps in {train_time_formatted:.3f} seconds')

----Training Started----
Step:0| pde_loss 0.00490 ic_loss 0.49357 bc_loss 0.00646
Step:500| pde_loss 0.02809 ic_loss 0.03244 bc_loss 0.00340
Step:1000| pde_loss 0.01272 ic_loss 0.00888 bc_loss 0.00096
Step:1500| pde_loss 0.00812 ic_loss 0.00491 bc_loss 0.00042
Step:2000| pde_loss 0.00466 ic_loss 0.00200 bc_loss 0.00038
Step:2500| pde_loss 0.00674 ic_loss 0.00189 bc_loss 0.00024
Step:3000| pde_loss 0.00529 ic_loss 0.00145 bc_loss 0.00017
Step:3500| pde_loss 0.00617 ic_loss 0.00121 bc_loss 0.00011
Step:4000| pde_loss 0.00517 ic_loss 0.00109 bc_loss 0.00016
Step:4500| pde_loss 0.00250 ic_loss 0.00060 bc_loss 0.00009
Step:5000| pde_loss 0.00186 ic_loss 0.00028 bc_loss 0.00027
Step:5500| pde_loss 0.00255 ic_loss 0.00039 bc_loss 0.00012
Step:6000| pde_loss 0.00209 ic_loss 0.00020 bc_loss 0.00002
Step:6500| pde_loss 0.00096 ic_loss 0.00017 bc_loss 0.00007
Step:7000| pde_loss 0.00082 ic_loss 0.00015 bc_loss 0.00002
Step:7500| pde_loss 0.00091 ic_loss 0.00018 bc_loss 0.00009
Step:8000| pde_loss

## Visualization

To visualize model solution, we obtain the output of the trained model and display it as a gif.

In [19]:
x = torch.linspace(x_min, x_max, samples)
t = torch.linspace(0, 1, samples)

u = [] 
time = []

for t_ in t:
    with torch.no_grad():
        X = torch.stack([x, torch.ones(samples)*t_], axis=-1)
        Y = mlp(X)
    u.append(Y.detach().numpy())
    time.append(t_)

def update(i):
    ax.clear()
    ax.plot(x, u[i])
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')
    ax.set_title(f'$t = {time[i]:.3f}$')
    ax.set_ylim(-1.1,1.1)
    ax.grid(True)
    return ax

fig = plt.figure(dpi=100)
ax = plt.subplot(1,1,1)
anim = animation.FuncAnimation(fig, update, frames=len(u), interval=200)
anim.save('burgers.gif', dpi=300, writer=PillowWriter(fps=25))
plt.close(fig)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/matplotlib/animation.py:892: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


[Animation](https://github.com/munozmfrancisco/PINNs/raw/main/GIFs/Forward/burgers.gif)